### This is the ipython transcript for the `RAPHE_reward.py`

In [1]:
%matplotlib inline

In [2]:
import os;
os.chdir('/home/gergely/code/analysis/analysis-scripts/')
import sys;
import copy;

import cPickle as pickle;

import pandas as pd;

import numpy as np;
import scipy.stats as stats;

import matplotlib;
matplotlib.use("pdf");

import matplotlib.pyplot as pp;
import matplotlib.backends.backend_pdf as pdf;
import matplotlib.gridspec as gs;

import lab;
import lab.analysis.imaging_analysis as ia;
import lab.analysis.behavior_analysis as ba;
import lab.analysis.reward_analysis as ra;

import lab.classes.exceptions as exc;

import lab.plotting.plotting_helpers as plotTools;

import VIP_common as vc;
matplotlib.rcParams['pdf.fonttype'] = 42;
matplotlib.rcParams['ps.fonttype'] = 42;

/home/gergely/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
/home/gergely/anaconda2/lib/python2.7/site-packages/seaborn/apionly.py:6: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)
/home/gergely/code/analysis/lab/misc/lfp_helpers.py:16: UserWarning: rhd package not found, lab.misc.lfp_helpers.py will not work
  warnings.warn("rhd package not found, lab.misc.lfp_helpers.py will not work")


In [3]:
def calcMaxConsecutive(boolArr):
    conSize=np.diff(
        np.where(
            np.concatenate(
                ([boolArr[0]], boolArr[:-1]!=boolArr[1:], [True])))[0]);
    if(conSize.size<=0):
        return 0;
    
    return np.max(conSize[::2]);

In [4]:
def bootstrapTraceDiff(df, groupbyCols, g1Key, g2Key, dataCol, timeCol, minSamples=5,
                       iterations=3000, ciLevel=0.95):
    groups=df.groupby(groupbyCols);
    if(len(groups)!=2):
        print("wrong number of groups");
        return pd.DataFrame();
    try:
        g1=groups.get_group(g1Key);
        g2=groups.get_group(g2Key);
    except KeyError:
        print("bad keys");
        return pd.DataFrame();
    
    g1NS=len(g1);
    g2NS=len(g2);
    
    numSamples=np.min([g1NS, g2NS]);
    if(numSamples<minSamples):
        return pd.DataFrame();
    
    ciPLow=(1-ciLevel)/2.0*100;
    ciPHigh=ciLevel*100.0+ciPLow;
    
    time=g1[timeCol].iloc[0];
    
    g1Data=np.vstack(g1[dataCol]);
    g2Data=np.vstack(g2[dataCol]);
    
    g1RSel=np.random.randint(g1NS, size=(iterations, numSamples));
    g2RSel=np.random.randint(g2NS, size=(iterations, numSamples));
    
    g1RS=np.nanmean(g1Data[g1RSel], 1);
    g2RS=np.nanmean(g2Data[g2RSel], 1);
    
    gDiffRS=g2RS-g1RS;
    
    gDiffRSPre=np.nanmean(gDiffRS[:, time<0], 1);
    gDiffRSPost=np.nanmean(gDiffRS[:, time>=0], 1);
    
    gDiffRSPPDiff=gDiffRSPost-gDiffRSPre;
    
    result=dict();
    
    result["diffTraceMed"]=[np.nanmedian(gDiffRS, 0)];
    result[timeCol]=[time];
    result["ppDiffTraceMed"]=np.nanmedian(gDiffRSPPDiff);
    result["ppDiffTraceCIH"]=np.nanpercentile(gDiffRSPPDiff, ciPHigh, interpolation="higher");
    result["ppDiffTraceCIL"]=np.nanpercentile(gDiffRSPPDiff, ciPLow, interpolation="lower");
    result["ppDiffTraceSig"]=((result["ppDiffTraceCIH"]<0)*-1)+((result["ppDiffTraceCIL"]>0)*1);
    
    return pd.DataFrame(result);

In [5]:
expSet=lab.ExperimentSet("raphe.sql");
mouseIDs=["gtr6_a1","gtr6_a3","gtr6_a4","gtr6_a5","gtr6_b1","gtr6_b2","gtr6_b3"];

hrExpGroups=vc.loadHRExpGrp(expSet, mouseIDs, minLicks=10);
ctxExpGroups=vc.loadExptsByType(expSet, mouseIDs, "contextDiscrimination", channel="Ch2")
rfExpGroups=vc.loadExptsByType(expSet, mouseIDs, "randomForaging", channel="Ch2");
# rtExpGroups=vc.loadExptsByType(expSet, mouseIDs, "runTraining", channel="Ch1")
# orfExpGroups=vc.loadExptsByType(expSet, mouseIDs, "operantRandomForaging");
# salExpGroup=vc.loadExptsByType(expSet, mouseIDs, "salience")["imaging"];

imExpList=list(rfExpGroups["imaging"])
imExpList.extend(list(hrExpGroups["imaging"]));
imExpList.extend(list(ctxExpGroups["imaging"]))
# imExpList.extend(list(rtExpGroups["imaging"]))
# imExpList.extend(list(orfExpGroups["imaging"]));

imExpGroup=lab.classes.ExperimentGroup(imExpList);

expProps=vc.loadExpPropsManualLabel(imExpGroup);
expProps["mouseID"]=map(lambda expt: expt.parent.get("mouseID"), expProps["expt"]);
expProps["expType"]=map(lambda expt: expt.get("experimentType"), expProps["expt"]);
expProps["frameP"]=map(lambda expt: expt.frame_period(), expProps["expt"]);
# expProps["Dir"]=map(lambda expt: expt.get("tSeriesDirectory"), expProps["expt"]);



In [6]:
expProps["keep"]=True;
expProps.set_index(["expType", "condition"], inplace=True);
# expProps.loc[("randomForaging", "A"), "keep"]=False;
expProps=expProps[np.array(expProps["keep"])].reset_index();

imExpGroup=lab.ExperimentGroup(list(expProps["expt"]));

In [7]:
expProps

,expType,condition,condition_day,condition_day_session,expt,mouseID,frameP,keep
0,randomForaging,A,A_0,A_0_0,[[]],gtr6_a1,0.033218,True
1,randomForaging,A,A_1,A_1_0,[[]],gtr6_a1,0.033346,True
2,randomForaging,A,A_0,A_0_0,[[]],gtr6_a5,0.033346,True
3,randomForaging,A,A_0,A_0_0,[[]],gtr6_b3,0.033218,True
4,hiddenRewards,A,A_0,A_0_0,[[]],gtr6_a3,0.033346,True
5,hiddenRewards,A,A_0,A_0_1,[[]],gtr6_a3,0.033346,True
6,hiddenRewards,A,A_0,A_0_2,[[]],gtr6_a3,0.033346,True
7,hiddenRewards,A,A_1,A_1_0,[[]],gtr6_a3,0.033346,True
8,hiddenRewards,A,A_1,A_1_1,[[]],gtr6_a3,0.033346,True
9,hiddenRewards,A,A_1,A_1_2,[[]],gtr6_a3,0.033346,True


In [8]:
stimuli=["running_stop", "water"];
data=vc.getData(imExpGroup, stimuli, channel="Ch2", imPreTime=3, imPostTime=3, 
                behaviorDataKeys=["water", "velocity", "licking"]);                

data=data.merge(expProps, on="expt");
data.drop(["ROI", "expt"], 1, inplace=True);
data["conseqTThresh"]=np.ceil(np.array(0.3/data["frameP"]));

data.rename(columns={"condensedROI": "cROI"}, inplace=True);

data["preRest"]=map(lambda v, t, tth: calcMaxConsecutive(v[t<-0.75]>1)<=tth, 
                    data["velocity"], data["velocityTime"], data["conseqTThresh"]);
data["postRest"]=map(lambda v, t, tth: calcMaxConsecutive(v[t>=0.75]>1)<=tth, 
                     data["velocity"], data["velocityTime"], data["conseqTThresh"]);
data["preRun"]=map(lambda v, t, tth: calcMaxConsecutive(v[t<-0.75]<1)<=tth, 
                   data["velocity"], data["velocityTime"], data["conseqTThresh"]);
data["postRun"]=map(lambda v, t, tth: calcMaxConsecutive(v[t>=0.75]<1)<=tth, 
                    data["velocity"], data["velocityTime"], data["conseqTThresh"]);
data["rewarded"]=map(lambda w: np.nansum(w)>0, data["water"]);
data["rewardedPre"]=map(lambda w, t: np.nansum(w[t<0])>0, data["water"], data["waterTime"]);
data["rewardedPost"]=map(lambda w, t: np.nansum(w[t>=0])>0, data["water"], data["waterTime"]);

stims=np.array(data["stimulus"]);
preRest=np.array(data["preRest"]);
postRest=np.array(data["postRest"]);
preRun=np.array(data["preRun"]);
postRun=np.array(data["postRun"]);

In [9]:
data["valid"]=(((stims=="running_start") & preRest & postRun) |
               ((stims=="running_stop") & preRun & postRest));

In [10]:
data.columns

Index([u'activity', u'stimStart', u'stimulus', u'time', u'cROI', u'water',
       u'lapNum', u'waterTime', u'velocity', u'velocityTime', u'licking',
       u'lickingTime', u'expType', u'condition', u'condition_day',
       u'condition_day_session', u'mouseID', u'frameP', u'keep',
       u'conseqTThresh', u'preRest', u'postRest', u'preRun', u'postRun',
       u'rewarded', u'rewardedPre', u'rewardedPost', u'valid'],
      dtype='object')

In [22]:
data.size

2487604

In [11]:
data["licked"]=map(lambda l, t: np.nansum(l[t>0])>0, data["licking"], data["lickingTime"]);
data["actDiff"]=map(lambda a, t: np.nanmean(a[t>=0])-np.nanmean(a[t<0]), 
                    data["activity"], data["time"]);

dataGroups=data.groupby(["stimulus", "rewarded", "licked"]);
rewardedData=dataGroups.get_group(("water", True, True)).append(
    dataGroups.get_group(("running_stop", True, True)));
rewardedData.loc[rewardedData["stimulus"]=="running_stop", "stimulus"]="running_stop_rewarded";
with open("/data/gergely/workingData/raphe/reward/rewarded_traces_new_gtr6Cohort.pkl", "wb") as fh:
    pickle.dump(rewardedData, fh);

/home/gergely/anaconda2/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:703: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


In [12]:
dataGroups.size()

stimulus      rewarded  licked
running_stop  False     False     18218
                        True       2344
              True      False       430
                        True       3536
water         True      False        33
                        True      64282
dtype: int64

In [13]:
data.set_index(["stimulus", "rewarded", "valid"], inplace=True);

In [33]:
temp = data.loc["running_stop"]
temp

activity  stimStart  \
rewarded valid                                                                 
False    False  [0.144394861848, 0.161523360185, 0.16932238406...        354   
         False  [0.0246826145902, 0.0338933072308, 0.035620649...        354   
         False  [0.114876770123, 0.13231105501, 0.167290935706...        354   
         False  [0.127039566612, 0.127795329735, 0.13246232034...        354   
         False  [0.0325244212433, 0.0461929044686, 0.061283932...        354   
         False  [0.0471826036711, 0.0461509366563, 0.058234286...        354   
         False  [0.218399887221, 0.22052611241, 0.213747493583...       1794   
         False  [0.160732782226, 0.150813034893, 0.14468866339...       1794   
         False  [0.150908355314, 0.159654549377, 0.17279388941...       1794   
         False  [0.161473119239, 0.159077890321, 0.14444207484...       1794   
         False  [0.173682171587, 0.174204702935, 0.17125168837...       1794   
         False  [0.141093983217, 0.127640510855, 0.10854841029...       1794   
         False  [0.404963071826, 0.422433531536, 0.42059010444...       2232   
         False  [0.303303515395, 0.294887572493, 0.27767199843...       2232   
         False  [0.539764255982, 0.563117073623, 0.56328099251...       2232   
         False  [0.499892983447, 0.476057435906, 0.48622514961...       2232   
         False  [0.370982654998, 0.353198863309, 0.33508422173...       2232   
         False  [0.291404891616, 0.286365887772, 0.28435378055...       2232   
         False  [-0.0644120726021, -0.0825606434391, -0.099896...       3083   
         False  [-0.0044208997339, -0.00877931582088, -0.01683...       3083   
         False  [-0.148528290773, -0.157036865286, -0.16100247...       3083   
         False  [-0.0702549744003, -0.0972185759514, -0.105064...       3083   
         False  [-0.0220755562712, -0.0376428843456, -0.041363...       3083   
         False  [-0.0478859709711, -0.0550862152593, -0.059154...       3083   
True     False  [-0.00465002971335, -0.0148012548603, -0.01433...       3941   
         False  [0.0128180699219, 0.0122833356037, 0.001912211...       3941   
         False  [-0.00693727636419, -0.0254805885036, -0.04388...       3941   
         False  [-0.0149385588349, 0.00607451477481, 0.0184259...       3941   
         False  [0.012709221644, 0.00376538279148, -0.00708486...       3941   
         False  [0.00140508590698, 0.00241271954858, 0.0010296...       3941   
...                                                           ...        ...   
False    False  [0.26834832749, 0.28181537024, 0.27372737836, ...      12944   
         False  [0.307659466947, 0.292433248444, 0.29108291069...      12944   
         False  [0.282401544153, 0.286499642291, 0.28205459712...      12944   
         False  [0.440773022188, 0.447463793298, 0.44472885603...      13306   
         False  [0.348377374928, 0.333300191719, 0.32616574874...      13306   
         False  [0.401597283542, 0.404810707561, 0.41526215659...      13306   
         False  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...      13496   
         False  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...      13496   
         False  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...      13496   
         False  [-0.0102558340174, 0.00426783180983, 0.0133782...      13740   
         False  [-0.028112219872, -0.0268538060762, -0.0293896...      13740   
         False  [-0.0576602989526, -0.0605612544002, -0.068627...      13740   
         True   [0.00834393049916, 0.0178184109858, 0.04441635...      14553   
         True   [-0.0141766914614, -0.000893266525356, 0.00780...      14553   
         True   [0.00989343570914, 0.0143213182468, 0.02219320...      14553   
         False  [0.282786484436, 0.292961583487, 0.30235236150...      14953   
         False  [0.2196393358, 0.243002847987, 0.259645342701,...      14953   
         False  [0.223051723629, 0.228462927287, 0.245548

In [14]:
runStopData=data.loc[("running_stop", True, True)].reset_index();
runStopData=runStopData.append(data.loc[("running_stop", True, False)].reset_index());
runStopData=runStopData.append(data.loc["running_stop", False, True].reset_index());

/home/gergely/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.
/home/gergely/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  
/home/gergely/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
runStopData

,stimulus,rewarded,valid,activity,stimStart,time,cROI,water,lapNum,waterTime,...,keep,conseqTThresh,preRest,postRest,preRun,postRun,rewardedPre,rewardedPost,licked,actDiff
0,running_stop,True,True,"[0.0261487084177, 0.0315281623608, 0.026204328...",6615,"[-2.967794, -2.934448, -2.901102, -2.867756, -...","(gtr6_b1, , 1544197520649)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3,"[-2.967794, -2.934448, -2.901102, -2.867756, -...",...,True,9.0,False,True,True,False,True,True,True,-0.079378
1,running_stop,True,True,"[-0.0343918480524, -0.034016061822, -0.0312674...",6615,"[-2.967794, -2.934448, -2.901102, -2.867756, -...","(gtr6_b1, , 1544197596897)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3,"[-2.967794, -2.934448, -2.901102, -2.867756, -...",...,True,9.0,False,True,True,False,True,True,True,-0.078511
2,running_stop,True,True,"[-0.00517899740006, 0.0015798462572, -0.004352...",6615,"[-2.967794, -2.934448, -2.901102, -2.867756, -...","(gtr6_b1, , 1544197623104)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3,"[-2.967794, -2.934448, -2.901102, -2.867756, -...",...,True,9.0,False,True,True,False,True,True,True,-0.021648
3,running_stop,True,True,"[0.00404972238242, -0.005497742786, -0.0127739...",6615,"[-2.967794, -2.934448, -2.901102, -2.867756, -...","(gtr6_b1, , 1544197658177)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3,"[-2.967794, -2.934448, -2.901102, -2.867756, -...",...,True,9.0,False,True,True,False,True,True,True,-0.009445
4,running_stop,True,True,"[-0.0375351253908, -0.0232472729318, -0.015789...",6615,"[-2.967794, -2.934448, -2.901102, -2.867756, -...","(gtr6_b1, , 1544197640818)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3,"[-2.967794, -2.934448, -2.901102, -2.867756, -...",...,True,9.0,False,True,True,False,True,True,True,-0.060489
5,running_stop,True,True,"[-0.016154770975, -0.0251485112233, -0.0318351...",6615,"[-2.967794, -2.934448, -2.901102, -2.867756, -...","(gtr6_b1, , 1544197291738)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3,"[-2.967794, -2.934448, -2.901102, -2.867756, -...",...,True,9.0,False,True,True,False,True,True,True,-0.045612
6,running_stop,True,True,"[0.03339257644, 0.0386569816417, 0.04610689624...",6615,"[-2.967794, -2.934448, -2.901102, -2.867756, -...","(gtr6_b1, , 1544197366856)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3,"[-2.967794, -2.934448, -2.901102, -2.867756, -...",...,True,9.0,False,True,True,False,True,True,True,-0.092420
7,running_stop,True,True,"[0.00268189694588, 0.00619400840821, -0.000653...",6615,"[-2.967794, -2.934448, -2.901102, -2.867756, -...","(gtr6_b1, , 1544197538153)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3,"[-2.967794, -2.934448, -2.901102, -2.867756, -...",...,True,9.0,False,True,True,False,True,True,True,-0.040937
8,running_stop,True,True,"[-0.0149352822918, -0.0187818032816, -0.019884...",6615,"[-2.967794, -2.934448, -2.901102, -2.867756, -...","(gtr6_b1, , 1544197571785)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3,"[-2.967794, -2.934448, -2.901102, -2.867756, -...",...,True,9.0,False,True,True,False,True,True,True,-0.048213
9,running_stop,True,True,"[0.0755101075045, 0.0727333012779, 0.075100845...",6615,"[-2.967794, -2.934448, -2.901102, -2.867756, -...","(gtr6_b1, , 1544197551536)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3,"[-2.967794, -2.934448, -2.901102, -2.867756, -...",...,True,9.0,False,True,True,False,True,True,True,-0.108334


In [15]:
runStopData.size

325140

In [60]:
runStopRDiffROIs=runStopData.groupby(["expType", "condition_day", "cROI"])

In [40]:
len(runStopData["expType"])

10838

In [50]:
group = runStopRDiffROIs
group.tail()

,stimulus,rewarded,valid,activity,stimStart,time,cROI,water,lapNum,waterTime,...,keep,conseqTThresh,preRest,postRest,preRun,postRun,rewardedPre,rewardedPost,licked,actDiff
623,running_stop,True,True,"[-0.0741801597586, -0.0903127111727, -0.079608...",4140,"[-2.982889, -2.95011, -2.917331, -2.884552, -2...","(gtr6_b3, , 1544213766670)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6,"[-2.982889, -2.95011, -2.917331, -2.884552, -2...",...,True,10.0,False,True,True,False,True,True,True,0.047675
624,running_stop,True,True,"[0.0176133076371, 0.0308661577334, 0.042648904...",4140,"[-2.982889, -2.95011, -2.917331, -2.884552, -2...","(gtr6_b3, , 1544213643502)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6,"[-2.982889, -2.95011, -2.917331, -2.884552, -2...",...,True,10.0,False,True,True,False,True,True,True,0.070572
625,running_stop,True,True,"[0.00892516619423, 0.0114608034013, 0.01493077...",4140,"[-2.982889, -2.95011, -2.917331, -2.884552, -2...","(gtr6_b3, , 1544213619159)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6,"[-2.982889, -2.95011, -2.917331, -2.884552, -2...",...,True,10.0,False,True,True,False,True,True,True,0.023763
626,running_stop,True,True,"[-0.0261475011167, -0.0233461274041, -0.011812...",4140,"[-2.982889, -2.95011, -2.917331, -2.884552, -2...","(gtr6_b3, , 1544213700530)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6,"[-2.982889, -2.95011, -2.917331, -2.884552, -2...",...,True,10.0,False,True,True,False,True,True,True,0.048490
627,running_stop,True,True,"[-0.023043593769, -0.0191034478007, -0.0132821...",4140,"[-2.982889, -2.95011, -2.917331, -2.884552, -2...","(gtr6_b3, , 1544213491233)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6,"[-2.982889, -2.95011, -2.917331, -2.884552, -2...",...,True,10.0,False,True,True,False,True,True,True,0.040803
628,running_stop,True,True,"[-0.106831529986, -0.0872518336068, -0.0854297...",4140,"[-2.982889, -2.95011, -2.917331, -2.884552, -2...","(gtr6_b3, , 1544213881177)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6,"[-2.982889, -2.95011, -2.917331, -2.884552, -2...",...,True,10.0,False,True,True,False,True,True,True,0.104707
629,running_stop,True,True,"[0.073216809136, 0.0564814171101, 0.0525529064...",4140,"[-2.982889, -2.95011, -2.917331, -2.884552, -2...","(gtr6_b3, , 1544213867751)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6,"[-2.982889, -2.95011, -2.917331, -2.884552, -2...",...,True,10.0,False,True,True,False,True,True,True,0.014393
630,running_stop,True,True,"[-0.0286334392204, -0.031332579573, -0.0260758...",4140,"[-2.982889, -2.95011, -2.917331, -2.884552, -2...","(gtr6_b3, , 1544213779223)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6,"[-2.982889, -2.95011, -2.917331, -2.884552, -2...",...,True,10.0,False,True,True,False,True,True,True,0.096250
631,running_stop,True,True,"[0.0035227352519, 0.0119693864972, 0.024365513...",4140,"[-2.982889, -2.95011, -2.917331, -2.884552, -2...","(gtr6_b3, , 1544213833625)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6,"[-2.982889, -2.95011, -2.917331, -2.884552, -2...",...,True,10.0,False,True,True,False,True,True,True,0.036213
632,running_stop,True,True,"[-0.0509472027171, -0.0365524280141, -0.021609...",4140,"[-2.982889, -2.95011, -2.917331, -2.884552, -2...","(gtr6_b3, , 1544213665121)","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",6,"[-2.982889, -2.95011, -2.917331, -2.884552, -2...",...,True,10.0,False,True,True,False,True,True,True,0.063027


In [32]:
runStopRDiffROIs=runStopData.groupby(["expType", "condition_day", "cROI"]).apply(
    bootstrapTraceDiff, ["rewarded"], False, True, "activity", "time", 
    minSamples=5, iterations=3000, ciLevel=0.99);

wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong number of groups
wrong numbe

In [33]:
temp=runStopData[["stimulus", "rewarded", "stimStart", "mouseID", "expType", "valid", "condition_day_session"]].drop_duplicates()

In [34]:
temp.groupby(["expType", "mouseID", "condition_day_session", "stimulus", "valid", "rewarded"]).size()

expType                mouseID  condition_day_session  stimulus      valid  rewarded
contextDiscrimination  gtr6_a3  A_0_0                  running_stop  False  True         8
                                                                     True   False        6
                                B_0_0                  running_stop  False  True         4
                                                                     True   True         1
                       gtr6_a5  A_0_0                  running_stop  False  True        11
                                                                     True   False       11
                                                                            True         5
                                B_0_0                  running_stop  False  True         5
                                                                     True   False        6
                       gtr6_b1  A_0_0                  running_stop  False  True        11
     

In [35]:
temp.groupby(["expType", "mouseID", "condition_day_session", "stimulus", "rewarded"]).size()

expType                mouseID  condition_day_session  stimulus      rewarded
contextDiscrimination  gtr6_a3  A_0_0                  running_stop  False        6
                                                                     True         8
                                B_0_0                  running_stop  True         5
                       gtr6_a5  A_0_0                  running_stop  False       11
                                                                     True        16
                                B_0_0                  running_stop  False        6
                                                                     True         5
                       gtr6_b1  A_0_0                  running_stop  False        6
                                                                     True        15
                                B_0_0                  running_stop  False       19
                                                                     True         

In [24]:
runStopSigROIs.head()

,expType,condition_day,cROI,rewarded,level_4,diffCIH,diffCIL,diffMed,diffSig,numTraces,postCIH,postCIL,postMed,preCIH,preCIL,preMed,act,time
0,randomForaging,A_1,"(gtr5_01, , 1533238970022)",False,0,0.037009,-0.047108,-0.006255,0.0,5.0,0.142726,-0.044452,0.038584,0.162394,-0.071127,0.044428,"[0.0686456637033, 0.0429531522244, 0.006013281...","[-2.83441, -2.66768, -2.50095, -2.33422, -2.16..."
1,randomForaging,A_1,"(gtr5_01, , 1533239014386)",False,0,-0.065179,-0.292066,-0.152782,-1.0,5.0,-0.044083,-0.199846,-0.117630,0.091684,-0.026606,0.035612,"[-0.0370432945658, -0.00518844859535, 0.045031...","[-2.83441, -2.66768, -2.50095, -2.33422, -2.16..."
2,randomForaging,A_1,"(gtr5_01, , 1533239157834)",False,0,0.046194,-0.115919,-0.037133,0.0,5.0,0.036924,-0.102518,-0.033551,0.047001,-0.052405,0.003581,"[0.00482485352016, -0.00225862192442, 0.015333...","[-2.83441, -2.66768, -2.50095, -2.33422, -2.16..."
3,randomForaging,A_1,"(gtr5_01, , 1533239257298)",False,0,0.036696,-0.136239,-0.069292,0.0,5.0,0.037930,-0.079483,-0.009452,0.114596,-0.028619,0.056903,"[-0.029991666698, -0.00228174662395, 0.0184941...","[-2.83441, -2.66768, -2.50095, -2.33422, -2.16..."
4,randomForaging,A_1,"(gtr5_01, , 1533239272640)",False,0,0.112448,-0.126598,0.002144,0.0,5.0,0.019105,-0.043625,-0.017559,0.105231,-0.130272,-0.019703,"[0.0541157622252, 0.00652556922304, 0.01178112...","[-2.83441, -2.66768, -2.50095, -2.33422, -2.16..."


In [54]:
temp=runStopData[["stimulus", "rewarded", "stimStart", "mouseID", "expType", "condition_day_session"]].drop_duplicates()

In [55]:
temp.groupby(["expType", "mouseID", "condition_day_session", "stimulus", "rewarded"]).size()

expType                mouseID  condition_day_session  stimulus      rewarded
contextDiscrimination  gtr6_a3  A_0_0                  running_stop  False        6
                                                                     True         8
                                B_0_0                  running_stop  True         5
                       gtr6_a5  A_0_0                  running_stop  False       11
                                                                     True        16
                                B_0_0                  running_stop  False        6
                                                                     True         5
                       gtr6_b1  A_0_0                  running_stop  False        6
                                                                     True        15
                                B_0_0                  running_stop  False       19
                                                                     True         

In [56]:
temp=runStopData[["stimulus", "rewarded", "stimStart", "mouseID", "expType", "valid", "condition_day_session"]].drop_duplicates()

In [57]:
temp.groupby(["expType", "mouseID", "condition_day_session", "stimulus", "valid", "rewarded"]).size()

expType                mouseID  condition_day_session  stimulus      valid  rewarded
contextDiscrimination  gtr6_a3  A_0_0                  running_stop  False  True         8
                                                                     True   False        6
                                B_0_0                  running_stop  False  True         4
                                                                     True   True         1
                       gtr6_a5  A_0_0                  running_stop  False  True        11
                                                                     True   False       11
                                                                            True         5
                                B_0_0                  running_stop  False  True         5
                                                                     True   False        6
                       gtr6_b1  A_0_0                  running_stop  False  True        11
     

there are enough events

In [36]:
runStopRDiffROIs.reset_index(inplace=True);
runStopRDiffROIs.rename(columns={"diffTraceMed": "actResidual", 
                                 "ppDiffTraceCIH": "residualDiffCIH",
                                 "ppDiffTraceCIL": "residualDiffCIL",
                                 "ppDiffTraceMed": "residualDiffMed",
                                 "ppDiffTraceSig": "residualDiffSig"},
                        inplace=True);

In [37]:
runStopSigROIs=runStopData.groupby(["expType", "condition_day", "cROI", "rewarded"]).apply(
    vc.calcResponseMetrics, col="activity", xCol="time", iterations=3000, ciLevel=0.99, 
    minDataSize=5, returnDF=True);
runStopSigROIs.reset_index(inplace=True);
runStopSigROIs.rename(columns={"diffSigResShuffle": "diffSig", 
                               "traceMed": "act", 
                               "traceXVal": "time"},
                      inplace=True);

In [22]:
runStopRDiffROIs.size

2370

In [31]:
runStopSigROIs.head()

,rewarded,expType,condition_day,cROI,level_4,diffCIH,diffCIL,diffMed,diffSig,numTraces,postCIH,postCIL,postMed,preCIH,preCIL,preMed,act,time
0,False,contextDiscrimination,A_0,"(gtr6_a3, , 1541471543988)",0,-0.013173,-0.110255,-0.063930,-1.0,6.0,-0.014220,-0.136260,-0.055109,0.047767,-0.043900,0.008280,"[0.0429335376773, 0.046285332704, 0.0537449466...","[-2.98962, -2.956402, -2.923184, -2.889966, -2..."
1,True,contextDiscrimination,A_0,"(gtr6_a3, , 1541471543988)",0,0.061686,-0.060523,-0.002215,0.0,8.0,0.014201,-0.078680,-0.027797,0.019484,-0.068225,-0.027242,"[0.0205699852033, 0.0279886125211, 0.028111839...","[-2.98962, -2.956402, -2.923184, -2.889966, -2..."
2,False,contextDiscrimination,A_0,"(gtr6_a3, , 1541471650955)",0,-0.001005,-0.073287,-0.041754,-1.0,6.0,-0.008404,-0.079020,-0.038338,0.027094,-0.025954,0.002098,"[0.0389348346667, 0.0419572103627, 0.048598735...","[-2.98962, -2.956402, -2.923184, -2.889966, -2..."
3,True,contextDiscrimination,A_0,"(gtr6_a3, , 1541471650955)",0,0.084748,0.013642,0.049926,1.0,8.0,0.041858,-0.009412,0.014379,-0.002634,-0.063750,-0.035800,"[0.0162087271307, 0.0125024574998, 0.010794214...","[-2.98962, -2.956402, -2.923184, -2.889966, -2..."
4,False,contextDiscrimination,A_0,"(gtr6_a3, , 1541471821564)",0,0.013800,-0.065651,-0.038456,0.0,6.0,0.004035,-0.077294,-0.034924,0.022840,-0.028442,0.002008,"[0.0101572821532, 0.0263501704258, 0.043737314...","[-2.98962, -2.956402, -2.923184, -2.889966, -2..."


In [25]:
runStopSigROIs.groupby(["expType", "condition_day", "rewarded"]).size()

expType                condition_day  rewarded
contextDiscrimination  A_0            False        27
                                      True         27
                       B_0            False        31
                                      True         13
hiddenRewards          A_0            False       100
                                      True         34
                       A_1            False       133
                                      True         53
                       B_0            False       108
                                      True         80
                       B_1            False       123
                                      True         70
randomForaging         A_0            False         6
                                      True         25
                       A_1            False        12
                                      True         12
dtype: int64

In [38]:
# not enough events. this is the remedy. the cells below should run
runStopSigROIs.set_index(["rewarded"], inplace=True)

In [39]:
plotROIs=runStopSigROIs.loc[False][["expType", "condition_day", "cROI", "diffSig", "act", "time"]].merge(
    runStopSigROIs.loc[True][["expType", "condition_day", "cROI", "diffSig", "act", "time"]], 
    on=["expType", "condition_day", "cROI"], suffixes=("URD", "RD"));

In [42]:
# this is extra line
runStopSigROIs.reset_index(inplace=True)
# until here

plotROIs=plotROIs.merge(runStopRDiffROIs[["expType", "condition_day", "cROI", 
                                          "actResidual", "time", "residualDiffSig"]],
                        on=["expType", "condition_day", "cROI"]);
                        
residualPlotTraces=plotROIs.groupby(["expType", "condition_day", 
                                     "residualDiffSig", "diffSigURD", "diffSigRD"]).apply(
                                         vc.aggregratePSTH, "actResidual", "time", 
                                         measureSuffix="", varMethod=stats.sem, varSuffix="SEM");
residualPlotTraces.rename(columns={"actResidual": "act",
                                   "actResidualSEM": "actSEM"}, inplace=True);
unRDPlotTraces=plotROIs.groupby(["expType", "condition_day", 
                                 "residualDiffSig", "diffSigURD", "diffSigRD"]).apply(
                                     vc.aggregratePSTH, "actURD", "timeURD", 
                                     measureSuffix="", varMethod=stats.sem, varSuffix="SEM");
unRDPlotTraces.rename(columns={"actURD": "act", "actURDSEM": "actSEM", "timeURD": "time"},
                      inplace=True);
rwdPlotTraces=plotROIs.groupby(["expType", "condition_day", 
                                 "residualDiffSig", "diffSigURD", "diffSigRD"]).apply(
                                     vc.aggregratePSTH, "actRD", "timeRD", 
                                     measureSuffix="", varMethod=stats.sem, varSuffix="SEM");
rwdPlotTraces.rename(columns={"actRD": "act", "actRDSEM": "actSEM", "timeRD": "time"},
                              inplace=True); 

/home/gergely/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)


In [43]:
plotTraces=dict();
plotTraces["residual"]=residualPlotTraces;
plotTraces["unRD"]=unRDPlotTraces;
plotTraces["rwd"]=rwdPlotTraces;
plotColors=dict();
plotColors["residual"]="red";
plotColors["unRD"]="black";
plotColors["rwd"]="blue";
traceKeys=["residual", "unRD", "rwd"];

In [44]:
expTypeList=residualPlotTraces.index.get_level_values(0).unique();
cdList=residualPlotTraces.index.get_level_values(1).unique();
sigList=[-1, 0, 1];
numSig=len(sigList);
sigInd=np.arange(numSig);

plotGridMain=gs.GridSpec(numSig, 1);

In [48]:
pages=pdf.PdfPages("/data/gergely/workingData/raphe/run_stop_rewarded_residual_gtr6Cohort.pdf");
for expType in expTypeList:
    for cd in cdList:
        fig=pp.figure(figsize=(numSig*2, numSig*numSig*2));
        fig.suptitle(str((expType, cd)));
        for resSig, i in zip(sigList, sigInd):
            plotGrid=gs.GridSpecFromSubplotSpec(numSig, numSig, 
                                                subplot_spec=plotGridMain[i, 0]);
            for rwdSig, j in zip(sigList, sigInd):
                for unRDSig, k in zip(sigList, sigInd):
                    ax=fig.add_subplot(plotGrid[j, k]);
                    numCells=0;
                    for traceKey in traceKeys:
                        try:
                            trace=plotTraces[traceKey].loc[(expType, cd,
                                                          resSig, unRDSig, rwdSig)].iloc[0];
                            numCells=trace["numSamples"];
                        except KeyError:
                            continue;
                        
                        ax.plot(trace["time"], trace["act"], lw=1, alpha=0.6, 
                                color=plotColors[traceKey]);
                        ax.fill_between(trace["time"], trace["act"]+trace["actSEM"],
                                        trace["act"]-trace["actSEM"], lw=0,
                                        alpha=0.4, color=plotColors[traceKey]);
                    
                    ax.axvline(0, lw=0.5, color="k", alpha=0.2, ls=(0, (5, 5)));
                    ax.set_ylim([-0.1, 0.3]);
                    ax.set_yticks([0, 0.1, 0.2, 0.3]);
                    ax.set_xticks([]);
                    ax.set_title(str(("rwd:"+str(rwdSig), 
                                      "unrwd:"+str(unRDSig), 
                                      "n="+str(numCells))));
                    plotTools.formatAxes(ax, "", "");
        
        pages.savefig(fig);
        pp.close(fig);
pages.close();

In [46]:
sigColors=["lightgrey", "r", "c"];
sigNums=plotROIs.groupby(["expType", "residualDiffSig", "diffSigRD", "diffSigURD"]).size();

In [47]:
pages=pdf.PdfPages("/data/gergely/workingData/raphe/run_stop_rewarded_residual_fractions_gtr6Cohort.pdf");
for expType in expTypeList:
    fig=pp.figure(figsize=(3, numSig*3));
    fig.suptitle(expType);
    for resSig, i in zip(sigList, sigInd):
        ax=fig.add_subplot(plotGridMain[i, 0]);
        
        for rwdSig, j in zip(sigList, sigInd):
            cellN=0;
            for unRDSig, k in zip(sigList, sigInd):
                try:
                    print((expType, resSig, rwdSig, unRDSig))
                    sigN=sigNums.loc[(expType, resSig, rwdSig, unRDSig)];
                except:
                    continue;
                
                ax.bar(j, sigN, width=0.6, bottom=cellN, 
                       color=sigColors[unRDSig], lw=0, align="center");
                cellN=cellN+sigN;
        
        ax.set_title("residual sig: "+str(resSig));
        ax.set_xlim([-0.5, numSig-0.5]);
        ax.set_xticks(sigInd);
        ax.set_xticklabels(map(lambda n: str(n), sigList));
        plotTools.formatAxes(ax, "rewarded RS sig", "num cells-days");
        
    pages.savefig(fig);
    pp.close(fig);
pages.close();

('contextDiscrimination', -1, -1, -1)
('contextDiscrimination', -1, -1, 0)
('contextDiscrimination', -1, -1, 1)
('contextDiscrimination', -1, 0, -1)
('contextDiscrimination', -1, 0, 0)
('contextDiscrimination', -1, 0, 1)
('contextDiscrimination', -1, 1, -1)
('contextDiscrimination', -1, 1, 0)
('contextDiscrimination', -1, 1, 1)
('contextDiscrimination', 0, -1, -1)
('contextDiscrimination', 0, -1, 0)
('contextDiscrimination', 0, -1, 1)
('contextDiscrimination', 0, 0, -1)
('contextDiscrimination', 0, 0, 0)
('contextDiscrimination', 0, 0, 1)
('contextDiscrimination', 0, 1, -1)
('contextDiscrimination', 0, 1, 0)
('contextDiscrimination', 0, 1, 1)
('contextDiscrimination', 1, -1, -1)
('contextDiscrimination', 1, -1, 0)
('contextDiscrimination', 1, -1, 1)
('contextDiscrimination', 1, 0, -1)
('contextDiscrimination', 1, 0, 0)
('contextDiscrimination', 1, 0, 1)
('contextDiscrimination', 1, 1, -1)
('contextDiscrimination', 1, 1, 0)
('contextDiscrimination', 1, 1, 1)
('hiddenRewards', -1, -1, -1